# 📄 PDF Translator - Full Gradio Version

**Translate scientific PDFs with the complete Gradio UI**

> 🔗 **GitHub:** [error-wtf/pdf-translator-enhanced](https://github.com/error-wtf/pdf-translator-enhanced)  
> Based on [thelanguagenerd/pdf-translator](https://github.com/thelanguagenerd/pdf-translator)

© 2025 Sven Kalinowski with small help of Lino Casu  
Licensed under the Anti-Capitalist Software License v1.4

---

## 🚀 How It Works

1. **Run all cells** - Installs Ollama, clones repo, installs dependencies
2. **Gradio launches** with a **public share link**
3. **Access from any device** - The link works for 72 hours
4. **Full features** - All 20 languages, formula protection, table detection

---

## ⚡ Features

- **🔬 Formula Protection** - 60+ LaTeX patterns protected during translation
- **🌍 20 Languages** - German, English, Arabic, Hebrew, Ukrainian, Japanese, Chinese, Hindi...
- **📊 Table Detection** - Preserves table structure
- **🖼️ Caption Anchoring** - Figures stay with captions
- **🔒 100% Local** - With Ollama local models, no data leaves Google's servers
- **☁️ Optional Ollama Cloud** - Use `...-cloud` models with internet + `ollama signin`

---

## 🤖 Recommended Models

| Model | Size | Description | VRAM |
|-------|------|-------------|------|
| **`openchat:7b`** ⭐ | 4.1 GB | ChatGPT-like, great for translations | 8 GB |
| `qwen2.5:7b` | 4.4 GB | Excellent multilingual | 8 GB |
| `llama3.1:8b` | 4.7 GB | Meta's latest | 10 GB |
| `neural-chat:7b` | 4.1 GB | Intel optimized | 8 GB |
| `qwen2.5:14b` | 9 GB | Best quality | 16 GB |

---

## GPU Runtime Setup

1. Click **Runtime** → **Change runtime type**
2. Select **T4 GPU** (free) or **A100** (Colab Pro)
3. Click **Save**

In [ ]:
#@title 1. Check GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv
import torch
print(f"\n✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🎮 GPU: {gpu_name}")
    print(f"💾 VRAM: {vram_gb:.1f} GB")
    if vram_gb >= 24:
        print("\n📌 Recommended: qwen2.5:14b or mistral-nemo:12b")
    elif vram_gb >= 16:
        print("\n📌 Recommended: llama3.1:8b or qwen2.5:7b")
    elif vram_gb >= 8:
        print("\n📌 Recommended: llama3.2:3b or phi3:mini")
    else:
        print("\n⚠️ Low VRAM - use OpenAI backend or smaller models")
else:
    print("\n❌ No GPU detected! Please enable GPU runtime:")
    print("   Runtime → Change runtime type → T4 GPU")

In [ ]:
#@title 2. Install Ollama
!curl -fsSL https://ollama.ai/install.sh | sh
print("\n✅ Ollama installed!")

In [ ]:
#@title 3. Start Ollama & Download Model
import subprocess
import time
import requests

# Start Ollama server
print("🚀 Starting Ollama server...")
subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(5)

# Check if running
try:
    r = requests.get("http://localhost:11434/api/tags", timeout=5)
    if r.status_code == 200:
        print("✅ Ollama server running!")
except:
    print("⚠️ Ollama not responding. Run this cell again.")

# Download model
# openchat:7b is a ChatGPT-like open source model, great for translations
model = "openchat:7b"  #@param ["openchat:7b", "gpt-oss:20b", "gpt-oss:20b-cloud", "gpt-oss:120b-cloud", "deepseek-v3.1:671b-cloud", "qwen3-coder:480b-cloud", "qwen2.5:7b", "llama3.1:8b", "qwen2.5:14b", "mistral:7b", "mistral-nemo:12b", "llama3.2:3b", "neural-chat:7b"]
print(f"\n📥 Downloading {model} (this may take a few minutes)...")
print("💡 openchat:7b is a ChatGPT-like open source model - great for translations!")
if model.endswith("-cloud"):
    print("\n☁️ Cloud model selected. Requires internet + Ollama v0.12+ and you must be signed in:")
    print("   ollama signin")

!ollama pull {model}
print(f"\n✅ Model {model} ready!")

In [ ]:
#@title 4. Clone Repository & Install Dependencies
%cd /content
!rm -rf pdf-translator-enhanced 2>/dev/null
!git clone https://github.com/error-wtf/pdf-translator-enhanced.git
%cd pdf-translator-enhanced

print("\n📦 Installing dependencies...")
!pip install -q -r requirements.txt
print("\n✅ All dependencies installed!")

In [ ]:
#@title 5. Launch Gradio App with Public Share Link 🚀
import os
os.chdir("/content/pdf-translator-enhanced")

print("=" * 60)
print("   📄 PDF Translator - Starting Gradio UI")
print("=" * 60)
print("")
print("🌐 A PUBLIC SHARE LINK will be generated below!")
print("📱 You can access it from any device for 72 hours.")
print("")
print("=" * 60)

# Patch gradio_app.py to use share=True
with open("gradio_app.py", "r", encoding="utf-8") as f:
    content = f.read()

# Replace launch() to include share=True
if "share=True" not in content:
    content = content.replace(
        "demo.launch()",
        "demo.launch(share=True)"
    )
    content = content.replace(
        "demo.launch(server_name=",
        "demo.launch(share=True, server_name="
    )
    with open("gradio_app.py", "w", encoding="utf-8") as f:
        f.write(content)

# Run the app
!python gradio_app.py